In [1]:
!pip install beautifulsoup4
!pip install requests_cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# Montamos nuestro google drive en el colab

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from gensim.models import Word2Vec

In [3]:
class EpochSaver: pass

In [4]:
from pathlib import Path
DATA_HOME = Path("/content/gdrive/My Drive/ml-practico-data")

In [5]:
model = Word2Vec.load(str(DATA_HOME / 'w2v/epoch_10'))

In [6]:
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import requests
import requests_cache

requests_cache.install_cache()

def get_name(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    return soup.select('.header .itemprop')[0].text

def get_image(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    candidates = soup.select('#name-poster')
    return candidates[0].attrs['src'] if candidates else 'https://m.media-amazon.com/images/G/01/imdb/images/nopicture/medium/name-2135195744._CB466677935_.png'

def render_person(id):
    name = get_name(id)
    picture = get_image(id)
    return f"""
    <div style="width: 150px; text-align: center">
        <h4 style='margin-top: -5px'>{name}</h4>
        <div style='font-size:75%; margin-bottom: 5px'>{id}</div>
        <a href="https://www.imdb.com/name/{id}" target="_blank">
            <img style="width: 100px; display: block; margin-left: auto; margin-right: auto;" src="{picture}"/>
        </a>
    </div>
    """

In [7]:
def show_similars(id, n=10):
    display(HTML(render_person(id)))
    renders = []
    for similar_id, score in model.wv.most_similar(id, topn=n):
        renders.append(render_person(similar_id))
        
    carousel = ''.join(
        [
            f'<div style="margin-left: 10px; float: left">{p}</div>' 
            for p in renders
        ]
    )
    display(HTML(f'<div style="width: 1800px">{carousel}</div>'))

In [8]:
show_similars('nm0001401')

In [9]:
show_similars('nm0201857')

In [10]:
show_similars('nm0006168')

In [12]:
show_similars('nm0655585')

In [13]:
show_similars('nm0000375')

In [14]:
show_similars('nm0511088')

In [15]:
show_similars('nm0000150')

# Algunos random

In [16]:
# Clonamos el repo para usar el codigo de la lib
!git clone https://github.com/elsonidoq/ml-practico-2022.git
!cd ml-practico-2022; git pull

fatal: destination path 'ml-practico-2022' already exists and is not an empty directory.
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 3), reused 6 (delta 3), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/elsonidoq/ml-practico-2022
   0c106cf..7c90893  main       -> origin/main
Updating 0c106cf..7c90893
Fast-forward
 notebooks/clase-3/01-rating-distribution.ipynb |  988 ++---
 notebooks/clase-3/04-crew-embeddings.ipynb     | 4694 ++++++++++++++++++++++--
 2 files changed, 4880 insertions(+), 802 deletions(-)


In [17]:
import sys
sys.path.append('ml-practico-2022')

from lib import data

In [18]:
import pandas as pd

title_basics = data.load_title_basics(DATA_HOME)
principals_df = pd.read_csv(DATA_HOME / 'title.principals.tsv.gz', compression='gzip', sep='\t')
ratings_df = pd.read_csv(DATA_HOME / 'title.ratings.tsv.gz', compression='gzip', sep='\t')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
candidates = (
    principals_df[
        principals_df.category.isin(['actress', 'actor', 'director']) &
        principals_df.tconst.isin(set(title_basics[title_basics.startYear >= 1980].tconst)) &
        principals_df.tconst.isin(set(ratings_df[ratings_df.numVotes > 100000].tconst))
    ].nconst.unique()
)

candidates = [c for c in candidates if c in model.wv.vocab]
len(candidates)

3788

In [21]:
from random import choice

for _ in range(10):
    id = choice(candidates)
    show_similars(id)